In [1]:
#Stacking
#ML Wave , Kaggle's guide
#구현하기

#기본원리

#classifier(regressor) 마다 각자 다른 강점(약점)이 있기 때문에
#같이 사용함으로서 정확도를 높일 수 있다.

#정확도 70 % 인 clf 3개 사용하면 
#34 % + 44 % = 77 % 의 정확도를 보여줄 수 있음.

#Kaggle guide

'''
1. Partition the training data into five test folds

2. Create a dataset called train_meta with the same row Ids and fold Ids as the training dataset,
with empty columns M1 and M2. Similarly create a dataset called test_meta 
with the same row Ids as the test dataset and empty columns M1 and M2

3. For each test fold
    3.1 Combine the other four folds to be used as a training fold
    
    3.2 For each base model 
    
        3.2.1 Fit the base model to the training fold and make predictions on the test fold.
        Store these predictions in train_meta to be used as features for the stacking model

4. Fit each base model to the full training dataset and make predictions on the test dataset.
Store these predictions inside test_meta

5. Fit a new model, S (i.e the stacking model) to train_meta,
using M1 and M2 as features. Optionally,
include other features from the original training dataset or engineered features

6. Use the stacked model S to make final predictions on test_meta
'''


'''
1. training data를 5개의 test fold로 쪼갬

2. train_meta 라는 데이터 셋으로 만듬 training_data로부터. M1,M2 라는 empty columns 만듬
유사하게 test_meta만듬. test 로부터. M1,M2 empty columns 만들고.

3. 각각의 test fold마다

    3.1 4개는 training fold로, 1개는 test fold
    
    3.2 각각의 clf model마다
    
        3.2.1 4개로 학습하고 1개로 prediction을 함. prediction은 train_meta에 features로 저장
        
4. 각각의 모델 fit하고 test 에 test_meta로 저장

5. 이과정  거친뒤에 남은 train_meta를 새 clf로 학습시킴

6. test_meta에도 동일과정.

'''

'\n1. training data를 5개의 test fold로 쪼갬\n\n2. train_meta 라는 데이터 셋으로 만듬 training_data로부터. M1,M2 라는 empty columns 만듬\n유사하게 test_meta만듬. test 로부터. M1,M2 empty columns 만들고.\n\n3. 각각의 test fold마다\n\n    3.1 4개는 training fold로, 1개는 test fold\n    \n    3.2 각각의 clf model마다\n    \n        3.2.1 4개로 학습하고 1개로 prediction을 함. prediction은 train_meta에 features로 저장\n        \n4. 각각의 모델 fit하고 test 에 test_meta로 저장\n\n5. 이과정  거친뒤에 남은 train_meta를 새 clf로 학습시킴\n\n6. test_meta에도 동일과정.\n\n'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
import xgboost as xgb
import itertools

In [ ]:
def rmsle_cv(model, n_folds=5):
    kf = KFold(n_folds, shuffle=True).get_n_splits(scale_data)
    rmse= np.sqrt(-cross_val_score(model, scale_data,logged_train_label, scoring="neg_mean_squared_error", cv = kf))
    return(rmse.mean())


In [ ]:
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.09,
                             learning_rate=0.009, 
                              n_estimators=100,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213,
                             random_state =7, nthread = -1)

xgb_m = xgb.XGBRegressor(leraning_rate = 0.05, n_estimators= 1000)

GBoost = GradientBoostingRegressor(max_depth=2, learning_rate=0.005,min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

KRR = KernelRidge(alpha=0.1, kernel='polynomial', degree=1, coef0=5)

ENet = ElasticNet(alpha=0.005, l1_ratio=.6, random_state=3)

lasso = Lasso(alpha =0.005, random_state=1)

RForest = RandomForestRegressor(n_estimators=100,max_depth=2,min_samples_leaf=2\
                               ,min_weight_fraction_leaf=0.5,max_leaf_nodes=2\
                                ,random_state=6)

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        
        for i, model in enumerate(self.base_models):
            
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                
                self.base_models_[i].append(instance)
                
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
      
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)


In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet,KRR,lasso,RForest),meta_model =  GBoost)
rmsle_cv(stacked_averaged_models)

In [ ]:
#base_models = (ENet,lasso,KRR,model_xgb),meta_model =  GBoost 0.053922348347839634
#base_models = (ENet,lasso,KRR,model_xgb),meta_model =  GBoost 0.054324193590506976
#base_models = (ENet,KRR),meta_model =  GBoost 0.0537208976739747